In [ ]:
"""
Testing out the full soma extraction

"""

In [ ]:
import cgal_Segmentation_Module as csm
from whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron
import time
import trimesh
import numpy as np
import datajoint as dj

In [ ]:
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")

In [ ]:
segment_id = 107816118160698192
version = 0
key = dict(segment_id=segment_id, version = version)

# Getting the Entire Mesh Processed: 

In [ ]:
#combine all the meshes into one mesh
def add_mesh_piece(main_mesh_vertices,main_mesh_faces,sub_mesh_vertices,sub_mesh_faces):
    """
    Purpose: Takes in a large mesh piece and an array of other meshes and 
    returns a large mesh with all meshes appended
    
    Parameters:
    main_mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates
    main_mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices
    sub_mesh_vertices(list of np.arrays) : list of np arrays with the vertices arrays for all subsegments to be added
    sub_mesh_faces(list of np.arrays) : list of np arrays with the faces arrays for all subsegments to be added
    
    Returns:
    mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates for NEW CONCATENATED MESH
    mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices for NEW CONCATENATED MESH
    
    
    Pseudocode: 
    - Checks: 
    a. Make sure there sub_mesh arrays are greater than 0 and of the same length

    1) Count the number of vertices and faces in the main mesh
    2) Iterate through the submesh vertices and faces. In loop:
    a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
    b. Add the vertices_count and add that to every number in the faces array
    c. Concatenate the submesh faces onto the larger mesh face
    d. Save this new vertices and faces as the main_mesh verts and faces
    e. Print out how many new vertices and faces added
    3) Print out number of segments added, total faces/vertices for new mesh
    4) Return the main mesh vertices and faces
    
    """
    #a. Make sure there sub_mesh arrays are greater than 0 and of the same length
    if len(sub_mesh_vertices) <= 0:
        print("There were no vertices in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) <= 0:
        print("There were no face in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) != len(sub_mesh_vertices):
        raise Exception("The sub_mesh_faces and sub_mesh_vertices length did not match")
        
    
    #1) Count the number of vertices and faces in the main mesh
    n_main_vertices = len(main_mesh_vertices)
    n_main_faces = len(main_mesh_faces)
    
    
    #2) Iterate through the submesh vertices and faces. In loop:
    for i,(sub_verts, sub_faces) in enumerate(zip(sub_mesh_vertices,sub_mesh_faces)):
        #a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
        n_sub_verts = len(sub_verts)
        n_sub_faces = len(sub_faces)
        
        main_mesh_vertices = np.vstack([main_mesh_vertices,sub_verts])

        
        #b. Add the vertices_count of main to every number in the faces array
        sub_faces = sub_faces + n_main_vertices
        
        #c. Concatenate the submesh faces onto the larger mesh face
        main_mesh_faces = np.vstack([main_mesh_faces,sub_faces])
        
        #d. Save this new vertices and faces as the main_mesh verts and faces (DONE)
        
        #e. Print out how many new vertices and faces added
        #print(f"Added subsegment {i} with {n_sub_verts} vertices and {n_sub_faces} faces")
        
        n_main_vertices = len(main_mesh_vertices)
        n_main_faces = len(main_mesh_faces)
    
    #3) Print out number of segments added, total faces/vertices for new mesh  
    print(f"Added {len(sub_mesh_vertices)} subsegements \n  --> final mesh: {len(main_mesh_vertices)} vertices and {len(main_mesh_faces)} faces")
        
    return main_mesh_vertices,main_mesh_faces 

In [ ]:
n = (m65.FromNeuromancer & key).fetch1()

In [ ]:
print("---Main Neuron ---")
print(n["segment_id"])
print(n["n_vertices"])
print(n["vertices"])
print(n["faces"])

#get all of the segments and their data
lookup_key = dict(segment_id=n["segment_id"],version = n["version"])
subsegments = (m65.FromNeuromancer.Subsegment & lookup_key).fetch(as_dict=True)

subsegment_dicts = dict([(k["subsegment_id"],dict(vertices=k["vertices"],faces=k["faces"])) for k in subsegments])

subsegment_ordered_list = np.sort(np.array(list(subsegment_dicts.keys())))
subsegments_vertices = [subsegment_dicts[k]["vertices"] for k in subsegment_ordered_list]
subsegments_faces = [subsegment_dicts[k]["faces"] for k in subsegment_ordered_list]


In [ ]:
# creating the entire mesh from the main mesh and all of its sub meshes: 
new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
                   main_mesh_faces=n["faces"],
                   sub_mesh_vertices = subsegments_vertices,
                   sub_mesh_faces=subsegments_faces)

# Running the Meshlab and CGAL algorithms to extract the mesh

In [ ]:
s = "_orig_inal.off"
indices = [i for i, a in enumerate(s) if a == "_"]
print(indices)
s[:-(len(s)-indices[-1])]

In [ ]:
import trimesh
original_main = trimesh.Trimesh(vertices=n["vertices"],faces=n["faces"])
output_mesh_name = "temp/" + str(n["segment_id"]) + "_original.off"
original_main.export("./" + output_mesh_name)
print("hello")

# having exporting the mesh then starts to do the algorithm

In [2]:
import cgal_Segmentation_Module as csm
from whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron
import time
import trimesh
import numpy as np
import datajoint as dj

import os
# total_test_meshes = [
# '103178515351946567_stitched.off',
# '104726695973782750_stitched.off',
# '106626583548494129_stitched.off',
# '107816118160698192_stitched.off',
# '110778132960975016_stitched.off',
# '96631955273149705_stitched.off',
# '81498689075439039_multiple_somas.off']

# output_file = total_test_meshes[0]
# folder_name = "test_neurons/" 

total_test_meshes = [
'81498689075439039_multiple_somas.off']

output_file = total_test_meshes[0]
folder_name = "neurons_potential_soma/" 

output_mesh_name = folder_name + output_file
print(f"Working on {output_file}")

indices = [i for i, a in enumerate(output_file) if a == "_"]
indices
seg_id_stripped = output_file[:indices[0]]
n = dict(segment_id=int(seg_id_stripped))


Working on 81498689075439039_multiple_somas.off


In [3]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    command_to_run = 'xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + script_command
    #command_to_run = 'meshlabserver ' + script_command
    
    print(command_to_run)
    subprocess_result = subprocess.run(command_to_run,shell=True)
    
    return subprocess_result

In [4]:
import os, contextlib
import pathlib
import subprocess
def meshlab_fix_manifold_path_specific_mls(input_path_and_filename,
                                           output_path_and_filename="",
                                           segment_id=-1,meshlab_script=""):
    #fix the path if it comes with the extension
    if input_path_and_filename[-4:] == ".off":
        path_and_filename = input_path_and_filename[:-4]
        input_mesh = input_path_and_filename
    else:
        raise Exception("Not passed off file")
    
    
    if output_path_and_filename == "":
        output_mesh = path_and_filename+"_mls.off"
    else:
        output_mesh = output_path_and_filename
    
    if meshlab_script == "":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    #print("meshlab_script = " + str(meshlab_script))
    #print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh


In [5]:
new_mesh = trimesh.load_mesh(output_mesh_name)
mesh_splits = new_mesh.split(only_watertight=False)

#len("Total mesh splits = " + str(mesh_splits))
#get the largest mesh
mesh_lengths = np.array([len(split.faces) for split in mesh_splits])

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set()
# sns.distplot(mesh_lengths)

largest_index = np.where(mesh_lengths == np.max(mesh_lengths))
largest_mesh = mesh_splits[largest_index][0]


stripped_ending = output_mesh_name[:-4]
pre_largest_mesh_path = stripped_ending + "_largest_piece.off"

largest_mesh.export(pre_largest_mesh_path)
print("done exporting")

done exporting


In [6]:
mesh_lengths

array([1404104,       4,       2,       4,       4,       2,       4,
             2,      30,       2,       4,       2,       4,       4,
             4,       4,       4,       4,       4,       4,       4,
             4,       2,       4,       4,       2,      18,       4,
             4,       4,       4,      20,      12,       4,       6,
             4,       4,       4,       2,       4,       4,       4,
             2,       2,       4,       4,       4,       4,       2,
            64,       4,      28,       4,       4,       4,       6,
             4,       4,       2,       2,       2,       2,       2,
             2,       2,       2,       2,       2,       2,       2,
             2,       2,       2,       2,       4,       2,       2,
             4,      20,       2,       4,       2,       4,       2,
             4,       4,       4,       4,       4,       4,       4,
             4,       2,       6,       4,       4,      34,       2,
             4,     

In [7]:
import pathlib
# run the meshlab server script
script_name = "poisson_working_meshlab.mls"
meshlab_script_path_and_name = str(pathlib.Path.cwd()) + "/" + script_name
input_path =str(pathlib.Path.cwd()) + "/" +  pre_largest_mesh_path

indices = [i for i, a in enumerate(input_path) if a == "_"]
stripped_ending = input_path[:-4]

output_path = stripped_ending + "_mls.off"
print(meshlab_script_path_and_name)
print(input_path)
print(output_path)
print("Running the mls function")
meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_path,
                                           output_path_and_filename=output_path,
                                           segment_id=n["segment_id"],
                                           meshlab_script=meshlab_script_path_and_name)

/notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Soma_Finder_2_26/poisson_working_meshlab.mls
/notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Soma_Finder_2_26/neurons_potential_soma/81498689075439039_multiple_somas_largest_piece.off
/notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Soma_Finder_2_26/neurons_potential_soma/81498689075439039_multiple_somas_largest_piece_mls.off
Running the mls function
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Soma_Finder_2_26/neurons_potential_soma/81498689075439039_multiple_somas_largest_piece.off -o /notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Soma_Finder_2_26/neurons_potential_soma/81498689075439039_multiple_somas_largest_piece_mls.off -s /notebooks3/Users/celii/Documents/Complete_Pinky

'/notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Soma_Finder_2_26/neurons_potential_soma/81498689075439039_multiple_somas_largest_piece_mls.off'

# The CGAL segmentation 

In [8]:
#import the mesh
new_mesh = trimesh.load_mesh(output_path)

In [9]:
mesh_splits = new_mesh.split(only_watertight=False)

len("Total mesh splits = " + str(mesh_splits))
#get the largest mesh
mesh_lengths = np.array([len(split.faces) for split in mesh_splits])

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set()
# sns.distplot(mesh_lengths)

largest_index = np.where(mesh_lengths == np.max(mesh_lengths))
largest_mesh = mesh_splits[largest_index][0]


indices = [i for i, a in enumerate(output_path) if a == "_"]
stripped_ending = output_path[:-4]
largest_mesh_path = stripped_ending + "_largest_piece.off"

largest_mesh.export(largest_mesh_path)
print("done exporting")

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

done exporting


In [10]:
from importlib import reload  # Python 3.4+ only.
import whole_neuron_classifier_datajoint_adapted as wcda 
wcda = reload(wcda)

In [11]:
segment_id = int(n["segment_id"])
faces = np.array(largest_mesh.faces)
verts = np.array(largest_mesh.vertices)
#run the whole algorithm on the neuron to test
verts_labels, faces_labels, soma_value,classifier = wcda.extract_branches_whole_neuron(import_Off_Flag=False,segment_id=segment_id,vertices=verts,
                     triangles=faces,pymeshfix_Flag=False,
                     import_CGAL_Flag=False,
                     return_Only_Labels=True,
                     clusters=3,
                     smoothness=0.2,
                    soma_only=True,
                    return_classifier = True
                    )


1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.0017414093017578125
2) Staring: Generating CGAL segmentation for neuron
Done writing OFF file

Starting CGAL segmentation
Right before cgal segmentation, clusters = 3, smoothness = 0.2
1
Finished CGAL segmentation algorithm: 178.86263251304626
2) Finished: Generating CGAL segmentation for neuron: 187.7117738723755
3) Staring: Generating Graph Structure and Identifying Soma
my_list_keys = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]
changed the median value
changed the mean value
changed the max value
changed the median value
changed the mean value
changed the median value
changed the max value
changed the mean value
changed the median value
c

In [46]:
from collections import Counter
labels_counter = Counter(faces_labels)

In [47]:
soma_faces = np.where(faces_labels == 5.0)[0]
soma_mesh = largest_mesh.submesh([soma_faces],append=True)
soma_mesh.export(folder_name + str(n["segment_id"]) + "_soma.off")
print("hello")

hello


In [48]:
non_soma_faces = np.where(faces_labels != 5.0)[0]
non_soma_mesh = largest_mesh.submesh([non_soma_faces],append=True)
non_soma_mesh.export(folder_name + str(n["segment_id"]) + "_NON_soma.off")
print("hello")

hello


# Exploring the CGAL classifier

In [27]:
labels_counter[44]

24175

In [15]:
classifier.sdf_final_dict[39]

{'median': 0.8127675000000001,
 'mean': 0.7745000080098222,
 'max': 0.890113,
 'n_faces': 17104}

In [12]:
median_values = np.array([v["median"] for k,v in classifier.sdf_final_dict.items()])
segmentation = np.array([k for k,v in classifier.sdf_final_dict.items()])

#order the compartments by greatest to smallest
sorted_medians = np.flip(np.argsort(median_values))
median_values[sorted_medians],segmentation[sorted_medians]

(array([0.8877695 , 0.8127675 , 0.277481  , 0.2248675 , 0.189898  ,
        0.1524405 , 0.124139  , 0.113944  , 0.106715  , 0.101004  ,
        0.099245  , 0.09496075, 0.0878234 , 0.08633795, 0.0831796 ,
        0.0830133 , 0.082234  , 0.0807923 , 0.08037115, 0.0788252 ,
        0.0757313 , 0.0753867 , 0.0654246 , 0.0639267 , 0.0628743 ,
        0.0626196 , 0.0624109 , 0.0605331 , 0.0594973 , 0.0567058 ,
        0.0517397 , 0.05089055, 0.050627  , 0.0498154 , 0.0491784 ,
        0.049157  , 0.0485628 , 0.04837385, 0.0482103 , 0.0481955 ,
        0.0475172 , 0.0473088 , 0.0471514 , 0.046291  , 0.045387  ,
        0.0439093 , 0.0431892 , 0.0431007 , 0.04263235, 0.0407222 ,
        0.04041675, 0.0391336 , 0.03849375, 0.0383993 , 0.03797785,
        0.03790585, 0.0379037 , 0.03788515, 0.0378785 , 0.03747355,
        0.0372941 , 0.03635565, 0.0355824 , 0.0351981 , 0.0346587 ,
        0.0336658 , 0.03292505]),
 array([22, 39, 21, 44, 43, 30, 33, 53,  7, 52, 26, 20, 63, 66, 40, 16, 24,
      

In [22]:
#visualize the soma in ipyvolume
import ipyvolume as ipv
import matplotlib.colors as cl
fig_2 = ipv.figure(figsize=(15,15))

main_mesh = largest_mesh

default_color = "green"

interest_label = [22,39,21]
color = ["red","red","red"]

#interest_label = []
#color = []

labels_list = classifier.labels_list


mesh = ipv.plot_trisurf(main_mesh.vertices[:,0],
                   main_mesh.vertices[:,1],
                   main_mesh.vertices[:,2],
                    triangles=main_mesh.faces
                   )
mesh.color = cl.to_rgb(default_color)


volume_max = np.max(main_mesh.vertices,axis=0)
volume_min = np.min(main_mesh.vertices,axis=0)

ranges = volume_max - volume_min
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_min[i] - buffer
        max_limits[i] = volume_max[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_min[i] - difference/2  - buffer
        max_limits[i] = volume_max[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])

ipv.style.set_style_light()
#ipv.style.box_off()
#ipv.style.axes_off()

ipv.show()

In [ ]:
labels_counter

In [35]:
labels_counter = Counter(classifier.labels_list)
significant_mesh_pieces = [k for k,v in labels_counter.items() if v > 10000]

In [36]:
significant_mesh_pieces

[0, 1, 4, 8, 10, 12, 13, 14, 17, 20, 22, 26, 34, 37, 39, 46, 50, 58]

In [43]:
median_values[58]

0.0639267

In [54]:
classifier.sdf_final_dict[58]

{'median': 0.0639267,
 'mean': 0.07691820562313163,
 'max': 0.2178,
 'n_faces': 19402}

In [55]:
missing_soma_mesh = largest_mesh.submesh(np.where(labels_list == 58),append=True)

In [50]:
#visualize the soma in ipyvolume
import ipyvolume as ipv
import matplotlib.colors as cl

fig1 = ipv.figure(figsize=(15,15))

main_mesh = largest_mesh

default_color = "green"


interest_label = [22,39, 58]
color = ["red"]*len(interest_label)

#interest_label = []
#color = []

labels_list = classifier.labels_list

"""
#This method did not work for coloring the faces
#compute the colors_list
current_color_list = np.tile(np.array(cl.to_rgb(default_color)),(len(main_mesh.faces),1))

for lab,c in zip(interest_label,color):
    faces_of_interest = np.where(labels_list == lab)[0]
    print("faces_of_interest = " + str(faces_of_interest))
    current_color_list[faces_of_interest,:] = np.tile(np.array(cl.to_rgb(c)),(len(faces_of_interest),1))

#set the figure size
ipv.figure(figsize=(15,15))
mesh = ipv.plot_trisurf(main_mesh.vertices[:,0],
                       main_mesh.vertices[:,1],
                       main_mesh.vertices[:,2],
                        triangles=main_mesh.faces,
                       color=current_color_list
                       )

"""

for lab,c in zip(interest_label,color):
    print("inside")
    current_mesh = largest_mesh.submesh(np.where(labels_list == lab),append=True)
    print("current_mesh size = " + str(len(current_mesh.vertices)))
    mesh_interested = ipv.plot_trisurf(current_mesh.vertices[:,0],
                       current_mesh.vertices[:,1],
                       current_mesh.vertices[:,2],
                        triangles=current_mesh.faces
                       )
    mesh_interested.color = cl.to_rgb(c)

#assemble mesh that was not in the labels of interest
not_interest_labels = [k for k in labels_list if k not in interest_label]
current_mesh = largest_mesh.submesh([not_interest_labels],append=True)
mesh_not_interested = ipv.plot_trisurf(current_mesh.vertices[:,0],
                   current_mesh.vertices[:,1],
                   current_mesh.vertices[:,2],
                    triangles=current_mesh.faces
                   )
mesh_not_interested.color = cl.to_rgb(default_color)


volume_max = np.max(main_mesh.vertices,axis=0)
volume_min = np.min(main_mesh.vertices,axis=0)

ranges = volume_max - volume_min
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_min[i] - buffer
        max_limits[i] = volume_max[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_min[i] - difference/2  - buffer
        max_limits[i] = volume_max[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])

ipv.style.set_style_light()
#ipv.style.box_off()
#ipv.style.axes_off()

ipv.show()

inside
current_mesh size = 10099
inside
current_mesh size = 8614
inside
current_mesh size = 9650


/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [ ]:
main_mesh

# Calculating the center of the soma

In [ ]:
soma_center = soma_mesh.vertices.mean(axis=0).astype("float")
print("Poor man's center from just averagin vertices = " + str(soma_center))
print("Trimesh center of mass = " + str(soma_mesh.center_mass))